In [44]:
from copy import copy
import inspect
from collections import UserDict

import ipywidgets as widgets
from IPython.display import display
import html, re

from attr import attributes

from ripple_down_rules.utils import is_iterable


def format_python_literal(value):
    s = repr(value)
    s = html.escape(s)
    s = re.sub(r'(&#x27;.*?&#x27;|".*?")', r"<span style='color:green'>\1</span>", s)
    s = re.sub(r'\b\d+(\.\d+)?\b', r"<span style='color:blue'>\g<0></span>", s)
    s = re.sub(r'\b(True|False|None)\b', r"<span style='color:darkorange'>\1</span>", s)
    return f"<code>{s}</code>"

def make_attr_row(attr, value, curr_depth=0, max_depth=3):
    """
    Create a row for displaying an attribute and its value.

    :param attr: The name of the attribute.
    :param value: The value of the attribute.
    :param curr_depth: The current depth of exploration.
    :param max_depth: The maximum depth to explore.
    """
    btn = widgets.Button(description=attr, layout=widgets.Layout(width='150px', flex='0 0 auto'))

    if attr == "scope":
        row = widgets.HBox([btn, widgets.HTML(value="<code>Scope</code>", layout=widgets.Layout(margin='0 0 0 10px'))])

    elif isinstance(value, (int, float, str, bool, type(None))):
        label = widgets.HTML(value=format_python_literal(value),
                             layout=widgets.Layout(margin='0 0 0 10px'))
        row = widgets.HBox([btn, label])
    elif hasattr(value, '__dict__') or isinstance(value, (list, dict, tuple, set)):
        nested = object_explorer(value, title=str(attr), curr_depth=curr_depth+1, max_depth=max_depth)
        label = widgets.HTML(value=format_python_literal(value),
                             layout=widgets.Layout(width='150px', flex='0 0 auto'))
        row = widgets.HBox([nested, label])
    else:
        label = widgets.HTML(value=format_python_literal(value),
                             layout=widgets.Layout(margin='0 0 0 10px'))
        row = widgets.HBox([btn, label])

    row.layout = widgets.Layout(flex='0 0 auto', width='100%')
    return row

def object_explorer(obj, title="Explorer", curr_depth=0, max_depth=3):
    """
    Create an interactive object explorer for Python objects.

    :param obj: The object to explore.
    :param title: The title of the explorer.
    :param curr_depth: The current depth of exploration (not used in this version).
    :param max_depth: The maximum depth to explore (not used in this version).
    """
    content = []
    if curr_depth > max_depth:
        content.append(widgets.HTML(value="<code>Max depth reached</code>"))
    elif isinstance(obj, (int, float, str, bool, type(None))):
        content.append(widgets.HTML(value=format_python_literal(obj),))
    elif is_iterable(obj) and not isinstance(obj, (dict, UserDict)):
        for i, item in enumerate(obj):
            content.append(make_attr_row(f"Item {i}", item, curr_depth, max_depth))
    elif isinstance(obj, dict):
        for key, value in obj.items():
            content.append(make_attr_row(str(key), value, curr_depth, max_depth))
    else:
        methods = []
        unreadable = []
        for attr in dir(obj):
            if attr.startswith('_'):
                continue
            try:
                val = getattr(obj, attr)
                if callable(val):
                    methods.append((attr, val))
                    continue
            except Exception as e:
                unreadable.append((attr, e))
                continue
            content.append(make_attr_row(attr, val, curr_depth, max_depth))
        for method in methods:
            content.append(make_attr_row(method[0], method[1], curr_depth, max_depth))
        for unreadable in unreadable:
            content.append(make_attr_row(unreadable[0], unreadable[1], curr_depth, max_depth))

    scrollable_vbox = widgets.VBox(content, layout=widgets.Layout(
        flex='0 0 auto',
        width='100%',
        align_items='stretch'
    ))

    scroll_area = widgets.Box([scrollable_vbox], layout=widgets.Layout(
        overflow='auto',          # ✅ enables scrollbar
        border='1px solid gray',
        height='300px',           # ✅ fixed height to force scroll
        width='100%',
        display='flex',           # ✅ required to make scrollbar show in Jupyter
        flex_flow='column',
        align_items='stretch'
    ))

    acc = widgets.Accordion(children=[scroll_area])
    acc.set_title(0, title)
    return acc


In [28]:
from ripple_down_rules.datastructures.dataclasses import CaseQuery
from ripple_down_rules.datasets import load_zoo_dataset, Species

cases, targets = load_zoo_dataset(cache_file="zoo")
cq = CaseQuery(cases[0], "species", (Species,), True , _target=targets[0])

In [45]:
class Sample:
    def __init__(self):
        self.name = "Robot"
        self.id = 42
        self.data = {"a": [1, 2], "b": "test"}

explorer = object_explorer(cq, title="CaseQuery")
display(explorer)

Stack(children=(Box(children=(VBox(children=(HBox(children=(Button(description='attribute_name', layout=Layout…